# Next steps 
- connect precip to DF,
- add new sites (e.g., regionval) to training DF with all the respective spatial resolution information
- connect regional data together to train model
- connect different regions
- add precipitation phase features (seasonal accumulated rain precip, seasonal accumulated snow precip as a function of temperature)
- explore adding other features stemming from SNOTEL, remote sensing (LULC), Snow Classifications (Sturms), energy balance

Put all units in SI, while it should not matter for model training since they are being normalized, they will be more interpretable.

In [57]:
region = 'S_Sierras'
output_res = 300
folder_path = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/{output_res}M_Resolution/GeoObsDFs"
files = [file for file in os.listdir(folder_path)]
files.sort()

files[60:]

['GeoObsdfs_20170816.parquet',
 'GeoObsdfs_20180304.parquet',
 'GeoObsdfs_20180422.parquet',
 'GeoObsdfs_20180423.parquet',
 'GeoObsdfs_20180425.parquet',
 'GeoObsdfs_20180426.parquet',
 'GeoObsdfs_20180528.parquet',
 'GeoObsdfs_20180601.parquet',
 'GeoObsdfs_20180602.parquet',
 'GeoObsdfs_20190309.parquet',
 'GeoObsdfs_20190315.parquet',
 'GeoObsdfs_20190316.parquet',
 'GeoObsdfs_20190317.parquet',
 'GeoObsdfs_20190324.parquet',
 'GeoObsdfs_20190325.parquet',
 'GeoObsdfs_20190326.parquet',
 'GeoObsdfs_20190329.parquet',
 'GeoObsdfs_20190417.parquet',
 'GeoObsdfs_20190418.parquet',
 'GeoObsdfs_20190421.parquet',
 'GeoObsdfs_20190427.parquet',
 'GeoObsdfs_20190428.parquet',
 'GeoObsdfs_20190501.parquet',
 'GeoObsdfs_20190502.parquet',
 'GeoObsdfs_20190503.parquet',
 'GeoObsdfs_20190604.parquet',
 'GeoObsdfs_20190605.parquet',
 'GeoObsdfs_20190608.parquet',
 'GeoObsdfs_20190609.parquet',
 'GeoObsdfs_20190611.parquet',
 'GeoObsdfs_20190613.parquet',
 'GeoObsdfs_20190614.parquet',
 'GeoObs

In [58]:
Vfiles = [file for file in os.listdir(folder_path)]
Vfiles.sort()
len(Vfiles)

99

In [7]:
# find out why throwing NaNs
# make sure all files are done

import os
import pandas as pd
from tqdm import tqdm
import numpy as np

HOME = os.path.expanduser('~')


region = 'S_Sierras'
output_res = 300
year = 2013
folder_path = f"{HOME}/SWEMLv2.0/data/Precipitation/{region}/{output_res}M_NLDAS_Precip/{year}"
Vfiles = [file for file in os.listdir(folder_path)]
Vfiles.sort()
#Vfiles

len(Vfiles)

# for file in Vfiles[0]:
#     x = pd.read_parquet(os.path.join(folder_path, file), engine='fastparquet')
#     print(file)

os.path.join(folder_path, Vfiles[0])

'/home/rjohnson18/SWEMLv2.0/data/Precipitation/S_Sierras/300M_NLDAS_Precip/2013/NLDAS_PPT_2013-04-03.parquet'

In [8]:
x = pd.read_parquet(os.path.join(folder_path, Vfiles[0]), engine='fastparquet')
x

TypeError: expected string or bytes-like object

In [63]:
date = pd.to_datetime('2013-04-03')
dayOfYear = date.strftime("%Y%j") 
dayOfYear

'2013093'

In [11]:
import pandas as pd
x = pd.read_parquet(os.path.join(folder_path, Vfiles[0]), engine='fastparquet')
#x = x[(x['swe_m']>0) & (x['hasSnow']==False)]
x.head(10)

,cell_id,Date,cen_lat,cen_lon,Elevation_m,Slope_Deg,Aspect_Deg,swe_m,nearest_site_1,nearest_site_2,nearest_site_3,nearest_site_4,nearest_site_5,nearest_site_6,VIIRS_SCA,hasSnow
0,S_Sierras_300M_38.185_-119.591,2013-04-03,38.185,-119.591,1742,6,18,0.585963,44.52,49.0,0.0,0.1,18.9,18.8,88.500000,True
1,S_Sierras_300M_38.185_-119.588,2013-04-03,38.185,-119.588,1760,5,11,0.570728,44.52,49.0,0.0,0.1,18.9,18.8,88.666667,True
2,S_Sierras_300M_38.182_-119.597,2013-04-03,38.182,-119.597,1706,7,36,0.668675,44.52,49.0,0.0,0.1,18.9,18.8,88.000000,True
3,S_Sierras_300M_38.182_-119.594,2013-04-03,38.182,-119.594,1717,6,0,0.675063,44.52,49.0,0.0,0.1,18.9,18.8,88.000000,True
4,S_Sierras_300M_38.182_-119.591,2013-04-03,38.182,-119.591,1742,6,18,0.518234,44.52,49.0,0.0,0.1,18.9,18.8,87.000000,True
5,S_Sierras_300M_38.182_-119.588,2013-04-03,38.182,-119.588,1760,5,11,0.522296,44.52,49.0,0.0,0.1,18.9,18.8,86.666667,True
6,S_Sierras_300M_38.182_-119.586,2013-04-03,38.182,-119.586,1784,11,5,0.513450,44.52,49.0,0.0,0.1,18.9,18.8,86.000000,True
7,S_Sierras_300M_38.18_-119.599,2013-04-03,38.180,-119.599,1697,4,56,0.679220,44.52,49.0,0.0,18.9,0.1,18.8,88.000000,True
8,S_Sierras_300M_38.18_-119.597,2013-04-03,38.180,-119.597,1706,7,36,0.593406,44.52,49.0,0.0,0.1,18.9,18.8,88.750000,True
9,S_Sierras_300M_38.18_-119.594,2013-04-03,38.180,-119.594,1717,6,0,0.605196,44.52,49.0,0.0,0.1,18.9,18.8,89.000000,True


In [92]:
x[x['VIIRS_SCA']>0]['VIIRS_SCA'].unique()

array([88., 89., 86., 90., 87., 84., 91., 85., 83., 81., 82., 79., 80.,
       77., 93., 76., 73., 69., 72., 78., 75., 74., 56., 92., 65., 71.,
       57., 63., 62., 94., 67., 68., 59., 64., 70., 61., 58., 66., 46.,
       53., 55., 51., 49., 60., 45., 41., 50., 44., 42., 47., 34., 33.,
       23., 30., 43., 19., 20., 31., 25., 52., 54., 26., 28., 16., 18.,
       35., 39., 38., 37., 36., 27., 24., 40., 17., 22., 15., 11., 12.,
       10., 29., 32., 21., 48., 14., 13.])

In [8]:
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
from rioxarray.merge import merge_arrays
import rasterstats as rs

files  = [f"{HOME}/SWEMLv2.0/data/VIIRS/WY2013/VNP10A1F_A2013093_h08v05_002_2023170194633_VIIRS_Grid_IMG_2D_CGF_NDSI_Snow_Cover_f55d280c.tif"]

tifs = [rxr.open_rasterio(file) for file in files]  # Open all the files as Rioxarray DataArrays

noLakes = [tif.where(tif != 237, other=0) for tif in tifs]  # replace all the lake values with 0
noOceans = [tif.where(tif != 239, other=0) for tif in noLakes]  # replace all the ocean values with 0
noErrors = [tif.where(tif <= 100, other=100) for tif in
            noOceans]  # replace all the other values with 100 (max Snow)
merge_arrays(noErrors, nodata=255)  # Merge the arrays


RasterioIOError: /home/rjohnson18/SWEMLv2.0/data/VIIRS/WY2013/VNP10A1F_A2013093_h08v05_002_2023170194633_VIIRS_Grid_IMG_2D_CGF_NDSI_Snow_Cover_f55d280c.tif: No such file or directory

In [ ]:
x_gdf = gpd.GeoDataFrame(x, geometry=gpd.points_from_xy(x.cen_lon, x.cen_lat), crs="EPSG:4326" )

x_gdf = x_gdf[x_gdf['hasSnow']==False]
x_gdf.head()

,cell_id,Date,cen_lat,cen_lon,Elevation_m,Slope_Deg,Aspect_Deg,swe_m,nearest_site_1,nearest_site_2,nearest_site_3,nearest_site_4,nearest_site_5,nearest_site_6,VIIRS_SCA,hasSnow,geometry
7,S_Sierras_300M_38.18_-119.599,2013-04-03,38.18,-119.599,1697,4,56,0.679220,44.52,49.0,0.0,18.9,0.1,18.8,NaN,False,POINT (-119.59900 38.18000)
8,S_Sierras_300M_38.18_-119.597,2013-04-03,38.18,-119.597,1706,7,36,0.593406,44.52,49.0,0.0,0.1,18.9,18.8,NaN,False,POINT (-119.59700 38.18000)
9,S_Sierras_300M_38.18_-119.594,2013-04-03,38.18,-119.594,1717,6,0,0.605196,44.52,49.0,0.0,0.1,18.9,18.8,NaN,False,POINT (-119.59400 38.18000)
10,S_Sierras_300M_38.18_-119.591,2013-04-03,38.18,-119.591,1742,6,18,0.589579,44.52,49.0,0.0,0.1,18.9,18.8,NaN,False,POINT (-119.59100 38.18000)
11,S_Sierras_300M_38.18_-119.588,2013-04-03,38.18,-119.588,1760,5,11,0.520599,44.52,49.0,0.0,0.1,18.9,18.8,NaN,False,POINT (-119.58800 38.18000)


In [6]:
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
from rioxarray.merge import merge_arrays
import rasterstats as rs


def createMergedRxr(files: list[str]) -> xr.DataArray:
    """
        Creates a merged (mosaic-ed) rasterio dataset from a list of files.

        Parameters:
            files (list[str]): A list of filepaths to open and merge.

        Returns:
            merged (DataArray): A merged DataArray.
    """

    # FIXME sometimes throws "CPLE_AppDefined The definition of geographic CRS EPSG:4035 got from GeoTIFF keys is not
    #   the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set
    #   GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF
    #   keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code."
    tifs = [rxr.open_rasterio(file) for file in files]  # Open all the files as Rioxarray DataArrays

    noLakes = [tif.where(tif != 237, other=0) for tif in tifs]  # replace all the lake values with 0
    noOceans = [tif.where(tif != 239, other=0) for tif in noLakes]  # replace all the ocean values with 0
    noErrors = [tif.where(tif <= 100, other=100) for tif in
                noOceans]  # replace all the other values with 100 (max Snow)
    return merge_arrays(noErrors, nodata=255)  # Merge the arrays

def augmentGeoDF(gdf: gpd.GeoDataFrame,
                 raster: xr.DataArray,
                 threshold: float = 20,  # TODO try 10
                 noData: int = 255,
                 buffer: float = None) -> gpd.GeoDataFrame:
    """
        Augments a GeoDataFrame with a raster's values.

        Parameters:
            gdf (GeoDataFrame): The GeoDataFrame to append the SCA to. Requires geometry to be an area, see buffer param
            raster (DataArray): The raster to augment the GeoDataFrame with.
            threshold (int): The threshold to use to determine if a pixel is snow or not.
            noData (int): The no data value of the raster.
            buffer (float): The buffer to use around the geometry. Set if the geometry is a point.

        Returns:
            gdf (GeoDataFrame): The augmented GeoDataFrame.
    """

    if buffer is not None:
        buffered = gdf.to_crs("3857").buffer(buffer,
                                             cap_style=3)  # Convert CRS to a projected CRS and buffer points into squares
        buffered = buffered.to_crs(raster.rio.crs)  # Convert the GeoDataFrame to the same CRS as the raster
    else:
        buffered = gdf.to_crs(raster.rio.crs)  # Convert the GeoDataFrame to the same CRS as the raster

    stats = rs.zonal_stats(buffered,  # pass the buffered geometry
                           raster.values[0],  # pass the raster values as a numpy array, TODO investigate passing GDAL
                           no_data=noData,
                           affine=raster.rio.transform(),  # required for passing numpy arrays
                           stats=['mean'],  # we only want the mean, others are available if needed
                           geojson_out=False,  # we will add the result back into a GeoDataFrame, so no need for GeoJSON
                           )

    gdf["VIIRS_SCA"] = [stat['mean'] for stat in stats]  # add the mean to the GeoDataFrame
    gdf["hasSnow"] = gdf["VIIRS_SCA"] > threshold

    return gdf


In [7]:
output_res = 300
threshold = 20
regional_raster = createMergedRxr(files) 
    

buffer = output_res

adf = augmentGeoDF(x_gdf, regional_raster, buffer=buffer, threshold=threshold)

RasterioIOError: /home/rjohnson18/SWEMLv2.0/data/VIIRS/WY2013/2012-2013NASA/VNP10A1F_A2013093_h08v05_002_2023170194633_VIIRS_Grid_IMG_2D_CGF_NDSI_Snow_Cover_f55d280c.tif: No such file or directory

In [99]:
adf.head()

,cell_id,Date,cen_lat,cen_lon,Elevation_m,Slope_Deg,Aspect_Deg,swe_m,nearest_site_1,nearest_site_2,nearest_site_3,nearest_site_4,nearest_site_5,nearest_site_6,VIIRS_SCA,hasSnow,geometry
7,S_Sierras_300M_38.18_-119.599,2013-04-03,38.18,-119.599,1697,4,56,0.679220,44.52,49.0,0.0,18.9,0.1,18.8,88.00,True,POINT (-119.59900 38.18000)
8,S_Sierras_300M_38.18_-119.597,2013-04-03,38.18,-119.597,1706,7,36,0.593406,44.52,49.0,0.0,0.1,18.9,18.8,88.75,True,POINT (-119.59700 38.18000)
9,S_Sierras_300M_38.18_-119.594,2013-04-03,38.18,-119.594,1717,6,0,0.605196,44.52,49.0,0.0,0.1,18.9,18.8,89.00,True,POINT (-119.59400 38.18000)
10,S_Sierras_300M_38.18_-119.591,2013-04-03,38.18,-119.591,1742,6,18,0.589579,44.52,49.0,0.0,0.1,18.9,18.8,89.00,True,POINT (-119.59100 38.18000)
11,S_Sierras_300M_38.18_-119.588,2013-04-03,38.18,-119.588,1760,5,11,0.520599,44.52,49.0,0.0,0.1,18.9,18.8,88.00,True,POINT (-119.58800 38.18000)


In [15]:
import pickle as pkl
nearest_snotel_dict_path = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/{output_res}M_Resolution"

#Load dictionary of nearest sites
print(f"Loading {output_res}M resolution grids for {region} region")
with open(f"{nearest_snotel_dict_path}/nearest_SNOTEL.pkl", 'rb') as handle:
    nearest_snotel = pkl.load(handle)

Loading 300M resolution grids for S_Sierras region


In [19]:
nearest_snotel['S_Sierras_300M_36.994_-119.624']

KeyError: 'S_Sierras_300M_36.994_-119.624'

In [3]:
cell_ids = aso_file.cell_id.unique()
siteave_dic = {}
from tqdm import tqdm

print(f"Getting unique cell ids and taking the spatial average to get {output_res} m resolution")
[average_duplicates(cell_id, aso_file, siteave_dic) for cell_id in tqdm(cell_ids[:5])]

Getting unique cell ids and taking the spatial average to get 300 m resolution


100%|██████████| 5/5 [00:00<00:00, 15.71it/s]


[None, None, None, None, None]

In [5]:
pd.concat(siteave_dic)

,,cell_id,cen_lat,cen_lon,swe_m
S_Sierras_300M_38.186_-119.59,S_Sierras_300M_38.186_-119.59,S_Sierras_300M_38.186_-119.59,38.186,-119.590,0.80
S_Sierras_300M_38.186_-119.589,S_Sierras_300M_38.186_-119.589,S_Sierras_300M_38.186_-119.589,38.186,-119.589,0.42
S_Sierras_300M_38.186_-119.591,S_Sierras_300M_38.186_-119.591,S_Sierras_300M_38.186_-119.591,38.186,-119.591,1.21
S_Sierras_300M_38.186_-119.588,S_Sierras_300M_38.186_-119.588,S_Sierras_300M_38.186_-119.588,38.186,-119.588,0.31
S_Sierras_300M_38.185_-119.592,S_Sierras_300M_38.185_-119.592,S_Sierras_300M_38.185_-119.592,38.185,-119.592,0.56


In [ ]:
aso_file.drop_duplicates(subset=['cell_id'], inplace=True)
aso_file

In [ ]:
cols = ['cen_lat', 'cen_lon']
ASO_meta_loc_DF = ASO_meta_loc_DF[cols]
ASO_meta_loc_DF[~ASO_meta_loc_DF.index.duplicated(keep='first')]


In [ ]:
round(37.080205, -3)

In [ ]:
Precippath = f"{HOME}/SWEMLv2.0/data/Precipitation/{region}/{output_res}M_NLDAS_Precip"
ppt = pd.read_hdf(f"{Precippath}/WY{year}.h5", key = '11N_cell_-119.59073383567106_38.18624284828164')
ppt.head()

In [ ]:
aso_gdf

In [ ]:

import os
import pandas as pd

HOME = os.path.expanduser('~')

region = 'S_Sierras'

#ASO observations
aso_swe_files_folder_path = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/Obsdf"

obs1 = pd.read_csv(f"{aso_swe_files_folder_path}/20130403_ObsDF.parquet")

obs1

In [ ]:
cols = [
    'cell_id', 'Date',  'cen_lat', 'cen_lon', 'geometry', 'Elevation_m', 'Slope_Deg',
    'Aspect_Deg', 'swe', 'nearest_site_1', 'nearest_site_2', 'nearest_site_3', 'nearest_site_4', 
    'nearest_site_5', 'nearest_site_6'
    ]